In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
import osmnx as ox
from Nummobility.core.TrajectoryDF import NumPandasTraj
from Nummobility.semantics.semantic_features import SemanticFeatures

from Nummobility.features.spatial_features import SpatialFeatures
import Nummobility.utilities.constants as const
from Nummobility.utilities.DistanceCalculator import FormulaLog


# # Reading the geolife dataset and converting to NumPandasTraj.
# # Also, lets, print the first 5 points of the dataset to
# # see how the dataframe looks.
# geolife = pd.read_csv('./data/geolife_sample.csv')
# geolife = NumPandasTraj(geolife,'lat','lon','datetime','id')
# geolife.head()

In [2]:
gulls = pd.read_csv('./data/gulls.csv')
gulls = NumPandasTraj(gulls,
                      latitude='location-lat',
                      longitude='location-long',
                      datetime='timestamp',
                      traj_id='tag-local-identifier',
                      rest_of_columns=[])
gulls.head()


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  
traj_id DateTime                                                                
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...

In [3]:
small = gulls.reset_index().loc[gulls.reset_index()['traj_id'] == '91732']
small = NumPandasTraj(small,
                      latitude='lat',
                      longitude='lon',
                      datetime='datetime',
                      traj_id='traj_id')
small.head()

event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  
traj_id DateTime                                                                
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...

In [4]:
%%time

tags = {'amenity': ['bank', 'atm']}


dist = SemanticFeatures.nearest_bank_from_point(coords=(43.643475, -79.379379),
                                                dist_threshold=10000,
                                                tags=tags)
dist.head()

CPU times: user 239 ms, sys: 4.5 ms, total: 243 ms
Wall time: 4.54 s


,osmid,amenity,brand,brand:wikidata,brand:wikipedia,name,website,geometry,addr:city,addr:housenumber,...,name:zh,nodes,building,building:levels,building:colour,building:material,roof:colour,roof:shape,designation,"Distance_from_(43.643475, -79.379379)"
0,2916414208,atm,NaN,NaN,NaN,Scotiabank,NaN,POINT (-79.37901 43.64400),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.445844


In [ ]:
%%time

bbox = SpatialFeatures.get_bounding_box(gulls)
print(bbox)
poi = ox.geometries_from_bbox(north=bbox[2],
                              south=bbox[0],
                              east=bbox[3],
                              west=bbox[1],
                              tags=tags)
poi

(-62.71617, 4.02517, 65.2325, 103.19317)


In [ ]:
bank_indicator = SemanticFeatures.bank_within_threshold(gulls,
                                                        poi,
                                                        10000)